In [ ]:
# NBVAL_SKIP
import os
os.environ['SPS_HOME'] = '/mnt/storage/annalena_data/sps_fsps'
#os.environ['SPS_HOME'] = '/home/annalena/sps_fsps'
#os.environ['SPS_HOME'] = '/Users/annalena/Documents/GitHub/fsps'

# RUBIX Pipeline for NIHAO

The RUBIX pipeline has been extended to support any simulation that can be handled via pynbody. We showcase this with the example of an NIHAO galaxy. This notebook demonstrates how to use the pipeline to transform NIHAO data into mock IFU cubes. Similar to Illustris, the pipeline executes data transformation in a linear process.

## How to Use the Pipeline
1. Define a config
2. Set up the pipeline yaml
3. Run the RUBIX pipeline
4. Analyze the mock data

## Step 1: Configuration

Below is an example configuration for running the pipeline with NIHAO data. Replace path and halo_path with the paths to your NIHAO snapshot and halo files. This configuration supports quick testing by using only a subset of the data (1000 particles).


In [ ]:
#NBVAL_SKIP
config = {
    "pipeline": {"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "NihaoHandler",
        "args": {
            "particle_type": ["stars", "gas"],
            "save_data_path": "data",
        },
        "load_galaxy_args": {"reuse": True},
        "subset": {"use_subset": False, "subset_size": 1000},
    },
    "simulation": {
        "name": "NIHAO",
        "args": {
            "path": "/mnt/storage/_data/nihao/nihao_classic/g8.26e11/g8.26e11.01024",
            "halo_path": "/mnt/storage/_data/nihao/nihao_classic/g8.26e11/g8.26e11.01024.z0.000.AHF_halos",
            #"path": "/home/annalena/g7.55e11/snap_1024/output/7.55e11.01024",
            #"halo_path": "/home/annalena/g7.55e11/snap_1024/output/7.55e11.01024.z0.000.AHF_halos",
            "halo_id": 0,
        },
    },
    "output_path": "output",

    "telescope": {
        "name": "MUSE_WFM",
        "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
        "lsf": {"sigma": 1.2},
        "noise": {"signal_to_noise": 100, "noise_distribution": "normal"},
    },
    "cosmology": {"name": "PLANCK15"},
    "galaxy": {
        "dist_z": 0.01,
        "rotation": {"type": "edge-on"},
    },
    "ssp": {
        "template": {"name": "FSPS"},
        "dust": {
                "extinction_model": "Cardelli89",
                "dust_to_gas_ratio": 0.01,
                "dust_to_metals_ratio": 0.4,
                "dust_grain_density": 3.5,
                "Rv": 3.1,
            },
    },
}

## Step 2: Pipeline YAML

To run the RUBIX pipeline, you need a YAML file (stored in rubix/config/pipeline_config.yml) that defines which functions are used during the execution of the pipeline.

## Step 3: Run the Pipeline

Now, simply execute the pipeline with the following code.


In [ ]:
#NBVAL_SKIP
from rubix.core.pipeline import RubixPipeline
pipe = RubixPipeline(config)

rubixdata = pipe.run()

## Step 4: Visualize the Mock Data
### Plot a Spectrum for a Single Spaxel

In [ ]:
#NBVAL_SKIP
import jax.numpy as jnp
import matplotlib.pyplot as plt

wave = pipe.telescope.wave_seq
spectra = rubixdata.stars.datacube

plt.plot(wave, spectra[12, 12, :])
plt.title("Spectrum of Spaxel [12, 12]")
plt.xlabel("Wavelength [Å]")
plt.ylabel("Flux")
plt.show()

### Create a Spatial Image from the Data Cube

In [ ]:
#NBVAL_SKIP
visible_indices = jnp.where((wave >= 4000) & (wave <= 8000))

visible_spectra = spectra[:, :, visible_indices[0]]
image = jnp.sum(visible_spectra, axis=2)

plt.imshow(image, origin="lower", cmap="inferno")
plt.colorbar(label="Integrated Flux")
plt.title("Spatial Image from Data Cube")
plt.show()

### Plotting the stellar age histogram

In [ ]:
#NBVAL_SKIP
from rubix.core.visualisation import stellar_age_histogram

stellar_age_histogram('./output/rubix_galaxy.h5')

### Mean line of sight velocity

In [ ]:
# NBVAL_SKIP
import numpy as np
import matplotlib.pyplot as plt

# Assuming your data arrays are defined as follows:
pixel_assignment = np.asarray(np.squeeze(rubixdata.stars.pixel_assignment))
velocities = np.asarray(rubixdata.stars.velocity[0, :, 2])

# Compute the sum of velocities and count per pixel using np.bincount
sum_velocity = np.bincount(pixel_assignment, weights=velocities)
counts = np.bincount(pixel_assignment)

# Calculate mean velocity; note: division by zero is avoided if every pixel has at least one star.
mean_velocity = sum_velocity / counts


# If you know the pixel grid dimensions (for example, a square grid)
n_pixels = len(mean_velocity)
grid_size = int(np.sqrt(n_pixels))
if grid_size * grid_size != n_pixels:
    raise ValueError("The total number of pixels is not a perfect square; please specify the grid shape explicitly.")

# Reshape the mean_velocity into a 2D array for imshow
velocity_map = mean_velocity.reshape((grid_size, grid_size))
print(velocity_map[12,12])

print(velocity_map[17,12]-velocity_map[7,12])
# Plot the result
plt.figure(figsize=(6, 5))
plt.imshow(velocity_map, origin='lower', interpolation='nearest', cmap='seismic')
plt.colorbar(label='Mean Velocity')
plt.title('Mean Velocity per Pixel')
plt.xlabel('X pixel index')
plt.ylabel('Y pixel index')
#storepath = f"output/datacube_NIHAO{config['data']['load_galaxy_args']['id']}_{config['pipeline']['name']}_velocity.png"
#plt.savefig(storepath)
plt.show()


### Mean stellar age

In [ ]:


# NBVAL_SKIP
import numpy as np
import matplotlib.pyplot as plt

# Assuming your data arrays are defined as follows:
pixel_assignment = np.asarray(np.squeeze(rubixdata.stars.pixel_assignment))
ages = np.asarray(rubixdata.stars.age[0, :])

# Compute the sum of velocities and count per pixel using np.bincount
sum_ages = np.bincount(pixel_assignment, weights=ages)
counts = np.bincount(pixel_assignment)

# Calculate mean velocity; note: division by zero is avoided if every pixel has at least one star.
mean_age = sum_ages / counts


# If you know the pixel grid dimensions (for example, a square grid)
n_pixels = len(mean_age)
grid_size = int(np.sqrt(n_pixels))
if grid_size * grid_size != n_pixels:
    raise ValueError("The total number of pixels is not a perfect square; please specify the grid shape explicitly.")

# Reshape the mean_velocity into a 2D array for imshow
age_map = mean_age.reshape((grid_size, grid_size))
print(age_map[12,12])

# Plot the result
plt.figure(figsize=(6, 5))
plt.imshow(age_map, origin='lower', interpolation='nearest', cmap='inferno')
plt.colorbar(label='Mean Age')
plt.title('Mean Age per Pixel')
plt.xlabel('X pixel index')
plt.ylabel('Y pixel index')
#storepath = f"./output/datacube_NIHAO{config['data']['load_galaxy_args']['id']}_{config["telescope"]["name"]}_{config['pipeline']['name']}_age.png"
#plt.savefig(storepath)
plt.show()





# DONE!

Congratulations, you have successfully processed NIHAO simulation data using the RUBIX pipeline.